In [1]:
import os
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import re
import random
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import KFold

INPUT_DIR = '../inputs/MiddleburyDataset/data/'
OUTPUT_DIR = '../results'

# 创建输出目录
os.makedirs(OUTPUT_DIR, exist_ok=True)

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"使用设备: {device}")

使用设备: mps


In [2]:
def read_calib_file(calib_file):
    """读取校准文件并提取参数"""
    with open(calib_file, 'r') as f:
        lines = f.readlines()
    
    params = {}
    
    for line in lines:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        
        key_value = line.split('=')
        if len(key_value) != 2:
            continue
        
        key, value = key_value
        
        # 处理相机矩阵
        if key == 'cam0' or key == 'cam1':
            # 示例格式: cam0=[1733.74 0 792.27; 0 1733.74 541.89; 0 0 1]
            value = value.replace('[', '').replace(']', '')
            matrix_values = []
            for row in value.split(';'):
                if row:
                    row_values = [float(v) for v in row.split()]
                    matrix_values.extend(row_values)
            params[key] = matrix_values
        # 处理其他数值参数
        else:
            try:
                params[key] = float(value)
            except ValueError:
                params[key] = value
    
    # 确保基本参数存在，如果缺失则使用默认值
    if 'width' not in params:
        params['width'] = 1920
    if 'height' not in params:
        params['height'] = 1080
    if 'ndisp' not in params:
        params['ndisp'] = 192
    
    return params

def read_pfm(file):
    """读取PFM文件"""
    with open(file, 'rb') as f:
        # Line 1: PF=>RGB (3 channels), Pf=>Greyscale (1 channel)
        header = f.readline().decode('utf-8').rstrip()
        if header == 'PF':
            color = True
        elif header == 'Pf':
            color = False
        else:
            raise Exception('Not a PFM file: ' + file)
        
        # Line 2: dimensions
        dim_match = re.match(r'^(\d+)\s(\d+)\s$', f.readline().decode('utf-8'))
        if dim_match:
            width, height = map(int, dim_match.groups())
        else:
            raise Exception('Malformed PFM header: ' + file)
        
        # Line 3: scale factor (negative for little-endian)
        scale = float(f.readline().decode('utf-8').rstrip())
        if scale < 0:  # little-endian
            endian = '<'
        else:
            endian = '>'  # big-endian
        
        # Data
        data = np.fromfile(f, endian + 'f')
        shape = (height, width, 3) if color else (height, width)
        data = np.reshape(data, shape)
        data = np.flipud(data)  # Flip vertically
        
        return data, scale

class MiddleburyDataset(Dataset):
    def __init__(self, root_dir, scenes, transform=None, phase='train', target_size=(256, 512)):
        """
        Args:
            root_dir (string): 包含所有场景的目录
            scenes (list): 要包含的场景名称列表
            transform (callable, optional): 可选的要应用于样本的变换
            phase (str): 'train' 或 'test'
            target_size (tuple): 调整后的目标尺寸 (height, width)
        """
        self.root_dir = root_dir
        self.scenes = scenes
        self.transform = transform
        self.phase = phase
        self.target_size = target_size
        self.samples = []
        
        # 收集所有有效样本
        for scene in scenes:
            scene_dir = os.path.join(root_dir, scene)
            
            # 获取校准文件路径
            calib_file = os.path.join(scene_dir, 'calib.txt')
            
            # 获取默认左右图像
            left_img_path = os.path.join(scene_dir, 'im0.png')
            right_img_path = os.path.join(scene_dir, 'im1.png')
            left_disp_path = os.path.join(scene_dir, 'disp0.pfm')
            
            if os.path.exists(left_img_path) and os.path.exists(right_img_path) and os.path.exists(left_disp_path) and os.path.exists(calib_file):
                # 读取校准信息
                calib_params = read_calib_file(calib_file)
                
                self.samples.append({
                    'left_img': left_img_path,
                    'right_img': right_img_path,
                    'left_disp': left_disp_path,
                    'scene': scene,
                    'calib_params': calib_params
                })
                
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        
        # 获取校准参数
        calib_params = sample['calib_params']
        original_width = int(calib_params['width'])
        original_height = int(calib_params['height'])
        max_disp = int(calib_params['ndisp'])
        
        # 加载图像
        left_img = Image.open(sample['left_img']).convert('RGB')
        right_img = Image.open(sample['right_img']).convert('RGB')
        
        # 应用变换到RGB图像
        if self.transform:
            left_img = self.transform(left_img)
            right_img = self.transform(right_img)
        
        # 加载视差图
        disparity, _ = read_pfm(sample['left_disp'])
        disparity = disparity.astype(np.float32)
        
        # 调整视差图大小 - 使用双线性插值
        disparity_pil = Image.fromarray(disparity)
        disparity_pil = disparity_pil.resize(self.target_size, Image.BILINEAR)
        disparity = np.array(disparity_pil)
        
        # 视差缩放因子 - 基于尺寸比例
        width_scale = self.target_size[1] / original_width
        disparity = disparity * width_scale
        
        # 转换视差为张量
        disparity = torch.from_numpy(disparity)
        
        # 返回带有校准参数的样本
        return {
            'left': left_img,
            'right': right_img,
            'disparity': disparity,
            'scene': sample['scene'],
            'calib_params': calib_params,
            'max_disp': max_disp
        }

def get_data_loaders(root_dir, fold_idx, k=5, batch_size=1):
    """
    为k折交叉验证创建训练和验证数据加载器
    
    Args:
        root_dir (str): 数据集路径
        fold_idx (int): 当前折索引 (0 to k-1)
        k (int): 折数
        batch_size (int): 批大小
    
    Returns:
        train_loader, val_loader, val_scenes, max_disp
    """
    # 列出所有场景
    scenes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    scenes.sort()  # 确保场景列表顺序一致
    
    print(f"找到 {len(scenes)} 个场景: {scenes[:5]}...")
    
    # 设置k折交叉验证
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    # 获取当前折的训练和验证索引
    train_indices = []
    val_indices = []
    
    for i, (train_idx, val_idx) in enumerate(kf.split(scenes)):
        if i == fold_idx:
            train_indices = train_idx
            val_indices = val_idx
            break
    
    train_scenes = [scenes[i] for i in train_indices]
    val_scenes = [scenes[i] for i in val_indices]
    
    print(f"Fold {fold_idx+1}: 训练场景数: {len(train_scenes)}, 验证场景数: {len(val_scenes)}")
    
    # 定义变换 - 明确指定目标尺寸
    target_size = (256, 512)
    transform = transforms.Compose([
        transforms.Resize(target_size),  # 调整大小为(256, 512)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # 创建数据集
    train_dataset = MiddleburyDataset(root_dir, train_scenes, transform=transform, phase='train', target_size=target_size)
    val_dataset = MiddleburyDataset(root_dir, val_scenes, transform=transform, phase='test', target_size=target_size)
    
    print(f"训练数据集大小: {len(train_dataset)}, 验证数据集大小: {len(val_dataset)}")
    
    # 查看第一个场景的校准信息
    if len(train_dataset) > 0:
        sample = train_dataset[0]
        calib_params = sample['calib_params']
        print(f"场景 '{sample['scene']}' 校准信息:")
        print(f"  原始尺寸: {int(calib_params['width'])}x{int(calib_params['height'])}")
        print(f"  最大视差: {int(calib_params['ndisp'])}")
        print(f"  视差范围: {calib_params.get('vmin', 'N/A')} - {calib_params.get('vmax', 'N/A')}")
        print(f"  基线长度: {calib_params.get('baseline', 'N/A')} mm")
        
        # 确定最大视差值
        max_disp = int(calib_params['ndisp'])
    else:
        max_disp = 192  # 默认值
    
    # 创建数据加载器 - 使用num_workers=0避免多进程问题
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    return train_loader, val_loader, val_scenes, max_disp

# 测试数据加载
def explore_dataset(data_dir):
    """探索数据集并可视化几个样本"""
    scenes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    scenes.sort()
    
    print(f"找到 {len(scenes)} 个场景")
    
    # 随机选择3个场景进行可视化
    sample_scenes = random.sample(scenes, min(3, len(scenes)))
    
    plt.figure(figsize=(15, 10))
    
    for i, scene in enumerate(sample_scenes):
        scene_dir = os.path.join(data_dir, scene)
        
        # 加载左右图像
        left_img = Image.open(os.path.join(scene_dir, 'im0.png'))
        right_img = Image.open(os.path.join(scene_dir, 'im1.png'))
        
        # 加载视差图
        disparity, _ = read_pfm(os.path.join(scene_dir, 'disp0.pfm'))
        
        # 将视差图转换为可视化图像
        disparity_normalized = disparity / np.max(disparity)
        
        # 绘制图像
        plt.subplot(3, 3, i*3+1)
        plt.imshow(left_img)
        plt.title(f'{scene} - Left Image')
        plt.axis('off')
        
        plt.subplot(3, 3, i*3+2)
        plt.imshow(right_img)
        plt.title(f'{scene} - Right Image')
        plt.axis('off')
        
        plt.subplot(3, 3, i*3+3)
        plt.imshow(disparity_normalized, cmap='plasma')
        plt.title(f'{scene} - Disparity Map')
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'dataset_samples.png'))
    plt.show()
    
    print(f"数据集样本已保存至 {os.path.join(OUTPUT_DIR, 'dataset_samples.png')}")

# 可视化一个批次
def visualize_batch(batch, output_dir, prefix="batch"):
    """可视化批次中的样本"""
    # 处理批次的第一个样本
    i = 0
    left_img = batch['left'][i].permute(1, 2, 0).cpu().numpy()
    right_img = batch['right'][i].permute(1, 2, 0).cpu().numpy()
    disp_img = batch['disparity'][i].cpu().numpy()
    scene_name = batch['scene'][i]
    
    # 反归一化图像
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    left_img = left_img * std + mean
    right_img = right_img * std + mean
    
    # 裁剪到[0,1]
    left_img = np.clip(left_img, 0, 1)
    right_img = np.clip(right_img, 0, 1)
    
    # 归一化视差图
    disp_min = disp_img.min()
    disp_max = disp_img.max()
    if disp_max > disp_min:
        disp_normalized = (disp_img - disp_min) / (disp_max - disp_min)
    else:
        disp_normalized = disp_img
    
    # 创建图像
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(left_img)
    plt.title(f'Left Image - {scene_name}')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(right_img)
    plt.title(f'Right Image - {scene_name}')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(disp_normalized, cmap='plasma')
    plt.title(f'Disparity Map - {scene_name}')
    plt.colorbar(label='Normalized Disparity')
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{prefix}_{scene_name}.png'))
    plt.close()
    
    print(f"已保存样本图像: {os.path.join(output_dir, f'{prefix}_{scene_name}.png')}")

In [3]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class FeatureExtraction(nn.Module):
    def __init__(self):
        super(FeatureExtraction, self).__init__()
        self.inplanes = 64
        
        # 第一个卷积层
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # ResNet块
        self.layer1 = self._make_layer(BasicBlock, 64, 3)
        self.layer2 = self._make_layer(BasicBlock, 128, 4, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 6, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 3, stride=2)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x1 = self.layer1(x)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)
        
        return x4

class CostVolume(nn.Module):
    def __init__(self, max_disp=192):
        super(CostVolume, self).__init__()
        self.max_disp = max_disp
        
    def forward(self, left_feature, right_feature):
        B, C, H, W = left_feature.size()
        cost_volume = torch.zeros(B, C*2, self.max_disp//4, H, W, device=left_feature.device)
        
        for i in range(self.max_disp//4):
            if i > 0:
                cost_volume[:, :C, i, :, i:] = left_feature[:, :, :, i:]
                cost_volume[:, C:, i, :, i:] = right_feature[:, :, :, :-i]
            else:
                cost_volume[:, :C, i, :, :] = left_feature
                cost_volume[:, C:, i, :, :] = right_feature
                
        cost_volume = cost_volume.contiguous()
        return cost_volume

class CostAggregation(nn.Module):
    def __init__(self, in_channels):
        super(CostAggregation, self).__init__()
        
        self.conv3d_1 = nn.Sequential(
            nn.Conv3d(in_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )
        
        self.conv3d_2 = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )
        
        self.conv3d_3 = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )
        
        self.conv3d_4 = nn.Sequential(
            nn.Conv3d(64, 1, kernel_size=3, padding=1),
        )
        
    def forward(self, x):
        x = self.conv3d_1(x)
        x = self.conv3d_2(x)
        x = self.conv3d_3(x)
        x = self.conv3d_4(x)
        return x

class DisparityRegression(nn.Module):
    def __init__(self, max_disp):
        super(DisparityRegression, self).__init__()
        self.max_disp = max_disp
        
    def forward(self, x):
        B, _, D, H, W = x.size()
        x = F.softmax(x.squeeze(1), dim=1)
        
        # 创建视差值 [0, 1, 2, ..., max_disp-1]
        disp_values = torch.arange(0, self.max_disp//4, dtype=torch.float32, device=x.device)
        disp_values = disp_values.view(1, D, 1, 1)
        
        # 计算期望的视差值
        disparity = torch.sum(x * disp_values, dim=1)
        
        # 缩放到全分辨率
        disparity = disparity * 4
        
        return disparity

class StereoNet(nn.Module):
    def __init__(self, max_disp=192):
        super(StereoNet, self).__init__()
        self.max_disp = max_disp
        
        # 特征提取
        self.feature_extraction = FeatureExtraction()
        
        # 成本体积构建
        self.cost_volume = CostVolume(max_disp)
        
        # 成本体积聚合
        self.cost_aggregation = CostAggregation(512*2)
        
        # 视差回归
        self.disparity_regression = DisparityRegression(max_disp)
        
    def forward(self, left, right):
        # 提取特征
        left_feature = self.feature_extraction(left)
        right_feature = self.feature_extraction(right)
        
        # 构建成本体积
        cost_volume = self.cost_volume(left_feature, right_feature)
        
        # 聚合成本
        cost = self.cost_aggregation(cost_volume)
        
        # 回归
        disparity = self.disparity_regression(cost)
        
        # 上采样到原始大小
        disparity = F.interpolate(disparity.unsqueeze(1), size=(left.size(2), left.size(3)), 
                                 mode='bilinear', align_corners=False).squeeze(1)
        
        return disparity

In [4]:
def train_model(model, train_loader, val_loader, fold_idx, max_disp=192, num_epochs=20):
    """训练立体模型"""
    model.to(device)
    
    # 损失函数
    criterion = nn.SmoothL1Loss()
    
    # 优化器
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # 学习率调度器
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    
    # 训练循环
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    val_disparities = []
    
    for epoch in range(num_epochs):
        # 训练
        model.train()
        epoch_loss = 0.0
        
        for sample in tqdm(train_loader, desc=f'Fold {fold_idx+1}, Epoch {epoch+1}/{num_epochs} (Training)'):
            left = sample['left'].to(device)
            right = sample['right'].to(device)
            target_disp = sample['disparity'].to(device)
            
            # 前向传播
            optimizer.zero_grad()
            output_disp = model(left, right)
            
            # 确保形状匹配 - 如果需要，调整目标视差图大小
            if output_disp.shape[1:] != target_disp.shape[1:]:
                target_disp = F.interpolate(target_disp.unsqueeze(1), 
                                           size=output_disp.shape[1:], 
                                           mode='bilinear',
                                           align_corners=False).squeeze(1)
            
            # 创建有效视差的掩码 - 使用每个样本的最大视差值
            batch_max_disp = max_disp
            mask = (target_disp > 0) & (target_disp < batch_max_disp)
            
            # 计算损失
            loss = criterion(output_disp[mask], target_disp[mask])
            
            # 反向传播和优化
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_train_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        # 验证
        model.eval()
        val_loss = 0.0
        val_disp_errors = []
        
        with torch.no_grad():
            for sample in tqdm(val_loader, desc=f'Fold {fold_idx+1}, Epoch {epoch+1}/{num_epochs} (Validation)'):
                left = sample['left'].to(device)
                right = sample['right'].to(device)
                target_disp = sample['disparity'].to(device)
                
                # 前向传播
                output_disp = model(left, right)
                
                # 确保形状匹配 - 如果需要，调整目标视差图大小
                if output_disp.shape[1:] != target_disp.shape[1:]:
                    target_disp = F.interpolate(target_disp.unsqueeze(1), 
                                               size=output_disp.shape[1:], 
                                               mode='bilinear',
                                               align_corners=False).squeeze(1)
                
                # 创建有效视差的掩码
                batch_max_disp = max_disp
                mask = (target_disp > 0) & (target_disp < batch_max_disp)
                
                # 计算损失
                loss = criterion(output_disp[mask], target_disp[mask])
                val_loss += loss.item()
                
                # 计算视差误差
                disp_error = torch.abs(output_disp[mask] - target_disp[mask]).mean().item()
                val_disp_errors.append(disp_error)
                
                # 保存最后一个epoch的预测视差
                if epoch == num_epochs - 1:
                    for i in range(output_disp.size(0)):
                        # 保存校准参数以便后续转换为深度
                        calib_params = {k: v for k, v in sample['calib_params'].items()}
                        
                        val_disparities.append({
                            'scene': sample['scene'][i],
                            'disparity': output_disp[i].cpu().numpy(),
                            'target': target_disp[i].cpu().numpy(),
                            'calib_params': calib_params
                        })
        
        avg_val_loss = val_loss / len(val_loader)
        avg_disp_error = np.mean(val_disp_errors)
        val_losses.append(avg_val_loss)
        
        print(f'Fold {fold_idx+1}, Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {avg_train_loss:.4f}')
        print(f'  Val Loss: {avg_val_loss:.4f}')
        print(f'  Val Disparity Error: {avg_disp_error:.4f} pixels')
        
        # 更新学习率
        scheduler.step()
        
        # 如果验证损失改善则保存模型
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, f'best_model_fold{fold_idx+1}.pth'))
    
    return val_disparities, avg_disp_error

def run_cross_validation(data_root, output_dir, k=5, num_epochs=20):
    """运行k折交叉验证"""
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 结果
    all_val_disparities = []
    fold_errors = []
    test_scenes = []
    
    # 运行k折交叉验证
    for fold_idx in range(k):
        print(f"Starting fold {fold_idx+1}/{k}")
        
        # 获取当前折的数据加载器
        train_loader, val_loader, val_scenes, max_disp = get_data_loaders(data_root, fold_idx, k=k)
        test_scenes.append(val_scenes)
        
        print(f"使用最大视差值: {max_disp}")
        
        # 初始化模型
        model = StereoNet(max_disp=max_disp)
        
        # 训练模型
        val_disparities, fold_error = train_model(model, train_loader, val_loader, fold_idx, max_disp, num_epochs)
        all_val_disparities.extend(val_disparities)
        fold_errors.append(fold_error)
        
        print(f"Fold {fold_idx+1} completed. Disparity Error: {fold_error:.4f} pixels")
    
    # 保存交叉验证结果
    cv_results = pd.DataFrame({
        'Fold': range(1, k+1),
        'Test_Scenes': [str(scenes) for scenes in test_scenes],
        'Disparity_Error': fold_errors
    })
    cv_results.to_csv(os.path.join(output_dir, 'ex4c_crossvalidation.csv'), index=False)
    
    # 生成样本视差图和深度图（前3个示例）
    for i, disp_data in enumerate(all_val_disparities[:3]):
        scene = disp_data['scene']
        pred_disp = disp_data['disparity']
        calib_params = disp_data['calib_params']
        
        # 获取相机参数
        baseline = float(calib_params.get('baseline', 100.0))
        focal_length = float(calib_params['cam0'][0])  # 焦距f在相机矩阵的[0,0]位置
        doffs = float(calib_params.get('doffs', 0.0))
        
        # 计算深度 Z = (baseline * f) / (d + doffs)
        # 避免除以零
        valid_disparity = np.where(pred_disp > 0.1, pred_disp, 0.1)
        depth = (baseline * focal_length) / (valid_disparity + doffs)
        
        # 归一化视差和深度以便可视化
        norm_disp = pred_disp / np.max(pred_disp)
        
        # 限制深度范围以便可视化（忽略极端值）
        depth_cap = np.percentile(depth, 95)  # 使用95百分位数作为上限
        depth_vis = np.clip(depth, 0, depth_cap)
        norm_depth = depth_vis / np.max(depth_vis)
        
        # 保存视差图
        plt.figure(figsize=(10, 6))
        plt.imshow(norm_disp, cmap='plasma')
        plt.colorbar(label='Normalized Disparity')
        plt.title(f'Predicted Disparity Map - Scene: {scene}')
        plt.savefig(os.path.join(output_dir, f'ex4b_{scene}_disparitymap.png'))
        plt.close()
        
        # 保存深度图
        plt.figure(figsize=(10, 6))
        plt.imshow(norm_depth, cmap='viridis')
        plt.colorbar(label='Normalized Depth (mm)')
        plt.title(f'Predicted Depth Map - Scene: {scene}')
        plt.savefig(os.path.join(output_dir, f'ex4b_{scene}_depthmap.png'))
        plt.close()
    
    print("Cross-validation completed. Results saved to", output_dir)

In [5]:
def visualize_disparity_and_depth(disparity, calib_params, scene_name, output_dir):
    """
    将视差图转换为深度图并可视化
    
    Args:
        disparity: 视差图
        calib_params: 校准参数
        scene_name: 场景名称
        output_dir: 输出目录
    """
    # 获取相机参数
    baseline = float(calib_params.get('baseline', 100.0))
    focal_length = float(calib_params['cam0'][0])  # 焦距f在相机矩阵的[0,0]位置
    doffs = float(calib_params.get('doffs', 0.0))
    
    # 计算深度 Z = (baseline * f) / (d + doffs)
    # 避免除以零
    valid_disparity = np.where(disparity > 0.1, disparity, 0.1)
    depth = (baseline * focal_length) / (valid_disparity + doffs)
    
    # 创建图像
    plt.figure(figsize=(15, 6))
    
    # 视差图
    plt.subplot(1, 2, 1)
    plt.imshow(disparity, cmap='plasma')
    plt.colorbar(label='Disparity (pixels)')
    plt.title(f'Disparity Map - {scene_name}')
    
    # 深度图
    plt.subplot(1, 2, 2)
    # 限制深度范围以便可视化
    depth_cap = np.percentile(depth, 95)  # 使用95百分位数作为上限
    depth_vis = np.clip(depth, 0, depth_cap)
    plt.imshow(depth_vis, cmap='viridis')
    plt.colorbar(label='Depth (mm)')
    plt.title(f'Depth Map - {scene_name}')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{scene_name}_disparity_depth.png'))
    plt.close()
    
    print(f"已保存{scene_name}的视差图和深度图")

In [6]:
def draw_box(draw, x, y, width, height, text, color):
    """绘制带文本的框"""
    # 转换颜色名称为RGB
    color_dict = {
        "lightblue": (173, 216, 230),
        "lightgreen": (144, 238, 144),
        "orange": (255, 165, 0),
        "lightsalmon": (255, 160, 122),
        "lightpink": (255, 182, 193),
        "gold": (255, 215, 0)
    }
    rgb_color = color_dict.get(color, (200, 200, 200))
    
    # 绘制矩形
    draw.rectangle([x, y, x+width, y+height], fill=rgb_color, outline=(0, 0, 0))
    
    # 绘制文本
    lines = text.split('\n')
    font = ImageFont.load_default()
    line_height = height // (len(lines) + 1)
    
    for i, line in enumerate(lines):
        text_width = len(line) * 6  # 近似文本宽度
        text_x = x + (width - text_width) // 2
        text_y = y + (i + 1) * line_height
        draw.text((text_x, text_y), line, fill=(0, 0, 0))

def draw_arrow(draw, x1, y1, x2, y2):
    """绘制从(x1, y1)到(x2, y2)的箭头"""
    draw.line([x1, y1, x2, y2], fill=(0, 0, 0), width=2)
    
    # 绘制箭头头部
    angle = np.arctan2(y2 - y1, x2 - x1)
    arrow_length = 10
    arrow_width = 5
    
    x_end = x2
    y_end = y2
    
    # 计算箭头头部的点
    x_arrow1 = x_end - arrow_length * np.cos(angle) + arrow_width * np.sin(angle)
    y_arrow1 = y_end - arrow_length * np.sin(angle) - arrow_width * np.cos(angle)
    
    x_arrow2 = x_end - arrow_length * np.cos(angle) - arrow_width * np.sin(angle)
    y_arrow2 = y_end - arrow_length * np.sin(angle) + arrow_width * np.cos(angle)
    
    # 绘制箭头头部
    draw.polygon([(x_end, y_end), (x_arrow1, y_arrow1), (x_arrow2, y_arrow2)], fill=(0, 0, 0))

def create_architecture_diagram(output_path=os.path.join(OUTPUT_DIR, 'ex4a_architecture.png')):
    """创建PSMNet架构的可视化"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 创建一个大的空白图像
    img_width = 1200
    img_height = 800
    img = Image.new('RGB', (img_width, img_height), color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    
    # 定义框的尺寸和位置
    box_width = 160
    box_height = 80
    
    # 绘制架构组件
    
    # 输入图像
    draw_box(draw, 100, 100, box_width, box_height, "Left Image\n(Input)", "lightblue")
    draw_box(draw, 100, 250, box_width, box_height, "Right Image\n(Input)", "lightblue")
    
    # 特征提取（共享权重）
    draw_box(draw, 350, 100, box_width, box_height, "Feature\nExtraction\n(ResNet)", "lightgreen")
    draw_box(draw, 350, 250, box_width, box_height, "Feature\nExtraction\n(Shared weights)", "lightgreen")
    
    # 绘制连接输入到特征提取的箭头
    draw_arrow(draw, 100+box_width, 100+box_height//2, 350, 100+box_height//2)
    draw_arrow(draw, 100+box_width, 250+box_height//2, 350, 250+box_height//2)
    
    # 成本体积
    draw_box(draw, 600, 175, box_width, box_height, "Cost Volume\nConstruction", "orange")
    
    # 绘制连接特征提取到成本体积的箭头
    draw_arrow(draw, 350+box_width, 100+box_height//2, 600, 175+box_height//2)
    draw_arrow(draw, 350+box_width, 250+box_height//2, 600, 175+box_height//2)
    
    # 3D CNN用于成本聚合
    draw_box(draw, 850, 175, box_width, box_height, "3D CNN\nCost Aggregation", "lightsalmon")
    
    # 绘制连接成本体积到成本聚合的箭头
    draw_arrow(draw, 600+box_width, 175+box_height//2, 850, 175+box_height//2)
    
    # 视差回归
    draw_box(draw, 850, 350, box_width, box_height, "Disparity\nRegression", "lightpink")
    
    # 绘制连接成本聚合到视差回归的箭头
    draw_arrow(draw, 850+box_width//2, 175+box_height, 850+box_width//2, 350)
    
    # 输出视差图
    draw_box(draw, 600, 500, box_width, box_height, "Disparity Map\n(Output)", "gold")
    
    # 绘制连接视差回归到输出的箭头
    draw_arrow(draw, 850+box_width//2, 350+box_height, 850+box_width//2, 500+box_height//2)
    draw_arrow(draw, 850+box_width//2, 500+box_height//2, 600+box_width, 500+box_height//2)
    
    # 标题和图例
    font = ImageFont.load_default()
    draw.text((450, 30), "PSMNet Architecture for Stereo Depth Estimation", fill=(0, 0, 0))
    
    # 保存图表
    img.save(output_path)
    print(f"Architecture diagram saved to {output_path}")

In [7]:
# 参数
k = 5  # 折数
num_epochs = 20  # 训练轮数

# 创建架构图
create_architecture_diagram()

# 测试读取校准文件
scenes = [d for d in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, d))]
if scenes:
    test_scene = scenes[0]
    calib_file = os.path.join(INPUT_DIR, test_scene, 'calib.txt')
    if os.path.exists(calib_file):
        print(f"\n读取样本校准文件: {calib_file}")
        calib_params = read_calib_file(calib_file)
        print("校准参数:")
        for key, value in calib_params.items():
            print(f"  {key}: {value}")

run_cross_validation(INPUT_DIR, OUTPUT_DIR, k=k, num_epochs=num_epochs)

Architecture diagram saved to ../results/ex4a_architecture.png

读取样本校准文件: ../inputs/MiddleburyDataset/data/artroom2/calib.txt
校准参数:
  cam0: [1734.04, 0.0, -133.21, 0.0, 1734.04, 542.27, 0.0, 0.0, 1.0]
  cam1: [1734.04, 0.0, -133.21, 0.0, 1734.04, 542.27, 0.0, 0.0, 1.0]
  doffs: 0.0
  baseline: 529.5
  width: 1920.0
  height: 1080.0
  ndisp: 190.0
  vmin: 55.0
  vmax: 160.0
Starting fold 1/5
找到 24 个场景: ['artroom1', 'artroom2', 'bandsaw1', 'bandsaw2', 'chess1']...
Fold 1: 训练场景数: 19, 验证场景数: 5
训练数据集大小: 19, 验证数据集大小: 5
场景 'artroom2' 校准信息:
  原始尺寸: 1920x1080
  最大视差: 190
  视差范围: 55.0 - 160.0
  基线长度: 529.5 mm
使用最大视差值: 190


Fold 1, Epoch 1/20 (Validation): 100%|██████████| 5/5 [00:01<00:00,  4.57it/s]


Fold 1, Epoch 1/20:
  Train Loss: 17.3057
  Val Loss: 11.3477
  Val Disparity Error: 11.8397 pixels


Fold 1, Epoch 2/20 (Validation): 100%|██████████| 5/5 [00:00<00:00, 10.70it/s]


Fold 1, Epoch 2/20:
  Train Loss: 13.1012
  Val Loss: 8.5108
  Val Disparity Error: 9.0006 pixels


...

Fold 5, Epoch 18/20:
  Train Loss: 9.6034
  Val Loss: 13.1464
  Val Disparity Error: 13.6432 pixels


Fold 5, Epoch 19/20 (Validation): 100%|██████████| 4/4 [00:00<00:00, 10.45it/s]


Fold 5, Epoch 19/20:
  Train Loss: 9.3876
  Val Loss: 8.6664
  Val Disparity Error: 9.1615 pixels


Fold 5, Epoch 20/20 (Validation): 100%|██████████| 4/4 [00:00<00:00, 10.58it/s]


Fold 5, Epoch 20/20:
  Train Loss: 8.5755
  Val Loss: 12.0303
  Val Disparity Error: 12.5215 pixels
Fold 5 completed. Disparity Error: 12.5215 pixels
Cross-validation completed. Results saved to ../results
